# Simulate depletion

This notebook process the CoNDERC data that contains both experimental and simulation.

Running this notebook also performs OpenMC depletion simulations for every experiment.

This can take over an hour on a typical laptop but is needed for production of all the results.

In [7]:
import copy
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import openmc
import pypact as pp  # needs latest version which can be installed with pip install git+https://github.com/fispact/pypact
from urllib.parse import urlparse
from urllib.request import urlopen, Request
import shutil
import tarfile
from zipfile import ZipFile
import os

from openmc_activator import OpenmcActivator

This downloads and extracts the CoNDERC data. This contains the FNS experimental data and FISPACT inputs and outputs.

In [ ]:
# download zip file
conderc_url = 'https://nds.iaea.org/conderc/fusion/files/fns.zip'
p = Request(conderc_url, headers={'User-Agent': 'Mozilla/5.0'})
with urlopen(p) as response, open('fns.zip', 'wb') as out_file:
    shutil.copyfileobj(response, out_file)

# unzip
with ZipFile('fns.zip', 'r') as f:
    f.extractall('.')

completed download
Unzipped


Read all the experiments from unzipped fns folder.

In [ ]:
here = Path('./fns')
assert(here.exists()), 'fns folder does not seem to exist. Run `download_fns_fusion_decay.py` first to download and unzip FNS benchmark files.'
experiments = {}
files = [q for q in here.glob('*') if q.is_dir()]
for f in files:
    if '_' in f.name: continue
    l = list(f.glob('*fluxes*'))
    experiments[f.name] = []
    for name in l:
        x = name.name.replace('_fluxes', '')
        experiments[f.name].append(x)

Reads the Fispact fluxes file that contains the neutron spectra

In [ ]:
# read flux data
flux_dict = {}
for k,l in experiments.items():
    flux_dict[k] = {}
    for exp in l:
        ff = pp.FluxesFile()
        pp.from_file(ff, here / k / (exp+'_fluxes'))
        assert(len(ff.values) == 709)
        ebins = ff.boundaries
        flux_dict[k][exp] = ff.values

Plot and example irradiation neutron spectra.

This example plots the neutron spectra used to irradiate silver (Ag) in the 2000 experimental campaign for 5 minutes of irradiation.

In [ ]:
# in this case we plot the silver Ag experiment spectrum but you could plot others
plt.stairs(values=flux_dict['Ag']['2000exp_5min'], edges=ebins)
plt.yscale('log')
plt.xscale('log')
plt.xlabel('Energy [eV]')  #TODO check these units
plt.ylabel('Flux [n/cm$^2$/s]')  #TODO check these units
plt.close()

Next we read in the experimental data so that it is in a more accessible form.
The times, data and uncertainties are read in.

In [ ]:
def read_experimental_data(exp_file):
    lines = open(exp_file).readlines()
    minutes = [q.strip().split()[0] for q in lines]
    # it's not always days actually, check other file
    vals = [q.strip().split()[1] for q in lines]
    unc = [q.strip().split()[2] for q in lines]
    return np.array(minutes, dtype=float), np.array(vals, dtype=float), np.array(unc, dtype=float)

# TODO consider replacing with pypact
def is_days(input_file):
    lines = open(input_file, 'r').readlines()
    lines = [q for q in lines if 'TIME' in lines]
    day_cnt = 0
    for line in lines:
        if 'DAYS' in line: day_cnt += 1
    if day_cnt == len(lines):
        return True
    elif day_cnt == 0:
        return False
    else:
        raise ValueError('Something is not right')

exp_data_dict = {'minutes': {}, 'data': {}, 'unc': {}}
for k,l in experiments.items():
    for k_ in exp_data_dict:
        exp_data_dict[k_][k] = {}
    for exp in l:
        exp_path = here / k / (exp+'.exp')
        exp_path = str(exp_path.absolute())
        input_path = here / k / ('TENDL-2017_' + exp + '.i')
        input_path = str(input_path.absolute())
        is_day = is_days(input_path)
        mins, vals, uncs = read_experimental_data(exp_path)
        if is_day:
            exp_data_dict['minutes'][k][exp] = mins * 60 * 24
        else:
            exp_data_dict['minutes'][k][exp] = mins
        exp_data_dict['data'][k][exp] = vals
        exp_data_dict['unc'][k][exp] = uncs
        assert(len(mins) == len(vals))

Now we get the irradiation setup including the flux and timesteps

In [ ]:
def read_irr_setup(filepath):
    ff = pp.InputData()
    pp.from_file(ff, filepath)
    cleaned_irradschedule = [item for item in ff._irradschedule if item != (0.0, 0.0)]
    flux_mag_list = [val[1] for val in cleaned_irradschedule] + [0.0] * len(ff._coolingschedule)
    days_list = np.cumsum([val[0] for val in cleaned_irradschedule] + ff._coolingschedule)/ (24*60*60)
    return days_list, flux_mag_list

def read_mat_setup(filepath):
    ff = pp.InputData()
    pp.from_file(ff, filepath)
    return ff._inventorymass.entries

def read_density(filepath):
    ff = pp.InputData()
    pp.from_file(ff, filepath)
    return ff._density

setup_dict = {'days': {}, 'flux_mag': {}, 'mass': {}, 'density': {}}
for k,l in experiments.items():
    for k_ in setup_dict:
        setup_dict[k_][k] = {}
    for exp in l:
        input_path = here / k / ('TENDL-2017_' + exp + '.i')
        input_path = str(input_path.absolute())
        days, flux_mag = read_irr_setup(input_path)
        mass_dict = {k:v/100 for k,v in read_mat_setup(input_path)}
        setup_dict['days'][k][exp] = days
        setup_dict['flux_mag'][k][exp] = flux_mag
        setup_dict['mass'][k][exp] = mass_dict
        setup_dict['density'][k][exp] = read_density(input_path)
        assert(len(days) == len(flux_mag))
        assert(isinstance(mass_dict, dict))
    
            
setup_dict['mg_flux'] = flux_dict
setup_dict['ebins'] = ebins

Now we can carry out depletion simulations in OpenMC

Set the chain file and cross sections to let OpenMC know where to find the data.

The nuclear data used can have an impact on how closely the results match.

To make this a fair comparison we recommend using the same nuclear data as the original Fispact simulations (Tendl 2017) and the chain file provided within the repository.

In [ ]:
# Setting the cross section path to the location used by the CI.
# If you are running this locally you will have to change this path to your local cross section path.
openmc.config['cross_sections'] = Path.home() / 'nuclear_data' / 'cross_sections.xml'

# Setting the chain file to the relative path of the chain file included in the repository.
# Also resolving the chain file to the absolute path which is needed till the next release of OpenMC.
openmc.config['chain_file'] = Path('./fns_spectrum.chain.xml').resolve()

Next we use the experiment descriptions to make OpenMC simulations

The irradiation duration, spectra, flux, material and mass are found from the IAEA Conderc benchmarks and passed to OpenMC functions to perform simulations of the experimental setup.

In [ ]:
openmc_result_dict = {}
all_activation_data = []
element_exp_names = []
for k, l in experiments.items():
    
    # this loop currently just simulates the Ag irradiation but can be easily
    # changed to do every nuclide in the benchmark suit by commenting the
    # line below.
    # if k != 'Ag': continue
    if k not in ['Cd', 'NiCr']: continue

    print(f'Running OpenMC for {k} {l}')
    
    if k not in openmc_result_dict:
        openmc_result_dict[k] = {}
    for exp in l:
        if exp in openmc_result_dict[k]:
            continue
        ccfe_flux = flux_dict[k][exp]
        # ebins is ccfs 709 flux bins
        # low to high
        # create new chain file

        # mass in grams
        mass_dict = setup_dict['mass'][k][exp]
        days_list = setup_dict['days'][k][exp]
        # days are cumulative, so we gotta provide diffs
        days_list = np.append(days_list[0], np.diff(days_list))
        flux_mag_list = setup_dict['flux_mag'][k][exp]

        # make openmc material
        mat = openmc.Material()
        for el, md in mass_dict.items():
            el = el.lower().capitalize()
            mat.add_element(el, md, percent_type='wo')
        mat.set_density('g/cm3', setup_dict['density'][k][exp])
        mat.depletable = True
        mat.temperature = 294
        tot_mass = sum(mass_dict.values())
        mat.volume = tot_mass / mat.density

        activation_data = {
            'materials': mat,
            'multigroup_flux': ccfe_flux,
            'energy': ebins,
            'source_rate': flux_mag_list,
            'timesteps': days_list
        }
        element_exp_names.append((k,exp))
        all_activation_data.append(activation_data)

Running OpenMC for Os ['2000exp_5min']
Running OpenMC for Cd ['2000exp_5min']
Running OpenMC for NiCr ['1996exp_7hour', '1996exp_5min', '2000exp_5min']
Running OpenMC for Rb ['2000exp_5min']


In [ ]:
obj = OpenmcActivator(
    activation_data=all_activation_data,
    timestep_units='d',
    chain_file='fns_spectrum.chain.xml',
)

all_metric_dict = obj.activate(metric_list=['mass', 'decay_heat'])

          294K


[openmc.deplete] t=0.0 s, dt=300.0 s, source=11160000000.0
[openmc.deplete] t=300.0 s, dt=35.000000000000014 s, source=0.0


/home/jon/.neutronicsworkshop2/lib/python3.11/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")


[openmc.deplete] t=335.0 s, dt=16.000000000000014 s, source=0.0
[openmc.deplete] t=351.0 s, dt=15.000000000000021 s, source=0.0
[openmc.deplete] t=366.0 s, dt=14.999999999999947 s, source=0.0
[openmc.deplete] t=380.99999999999994 s, dt=15.000000000000021 s, source=0.0
[openmc.deplete] t=395.99999999999994 s, dt=25.999999999999993 s, source=0.0
[openmc.deplete] t=421.99999999999994 s, dt=36.00000000000001 s, source=0.0
[openmc.deplete] t=457.99999999999994 s, dt=36.00000000000001 s, source=0.0
[openmc.deplete] t=493.99999999999994 s, dt=51.999999999999986 s, source=0.0
[openmc.deplete] t=545.9999999999999 s, dt=65.99999999999997 s, source=0.0
[openmc.deplete] t=611.9999999999999 s, dt=66.00000000000006 s, source=0.0
[openmc.deplete] t=678.0 s, dt=93.99999999999996 s, source=0.0
[openmc.deplete] t=772.0 s, dt=127.00000000000001 s, source=0.0
[openmc.deplete] t=899.0 s, dt=126.00000000000006 s, source=0.0
[openmc.deplete] t=1025.0 s, dt=186.99999999999994 s, source=0.0
[openmc.deplete] t=

/home/jon/.neutronicsworkshop2/lib/python3.11/site-packages/openmc/deplete/results.py:94: FutureWarning: The ResultsList.from_hdf5(...) method is no longer necessary and will be removed in a future version of OpenMC. Use Results(...) instead.
  warn(


[openmc.deplete] t=0.0 s, dt=300.0 s, source=11160000000.0
[openmc.deplete] t=300.0 s, dt=20.00000000000003 s, source=0.0


/home/jon/.neutronicsworkshop2/lib/python3.11/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")


[openmc.deplete] t=320.0 s, dt=14.999999999999984 s, source=0.0
[openmc.deplete] t=335.0 s, dt=14.999999999999984 s, source=0.0
[openmc.deplete] t=350.0 s, dt=16.000000000000053 s, source=0.0
[openmc.deplete] t=366.00000000000006 s, dt=14.999999999999947 s, source=0.0
[openmc.deplete] t=381.0 s, dt=15.000000000000021 s, source=0.0
[openmc.deplete] t=396.0 s, dt=24.99999999999996 s, source=0.0
[openmc.deplete] t=420.99999999999994 s, dt=37.000000000000036 s, source=0.0
[openmc.deplete] t=458.0 s, dt=36.00000000000001 s, source=0.0
[openmc.deplete] t=494.0 s, dt=48.999999999999964 s, source=0.0
[openmc.deplete] t=543.0 s, dt=66.00000000000006 s, source=0.0
[openmc.deplete] t=609.0 s, dt=65.99999999999997 s, source=0.0
[openmc.deplete] t=675.0 s, dt=93.99999999999996 s, source=0.0
[openmc.deplete] t=769.0 s, dt=126.00000000000006 s, source=0.0
[openmc.deplete] t=895.0 s, dt=123.00000000000004 s, source=0.0
[openmc.deplete] t=1018.0 s, dt=183.99999999999994 s, source=0.0
[openmc.deplete] t

/home/jon/.neutronicsworkshop2/lib/python3.11/site-packages/openmc/deplete/results.py:94: FutureWarning: The ResultsList.from_hdf5(...) method is no longer necessary and will be removed in a future version of OpenMC. Use Results(...) instead.
  warn(


In [ ]:
for entry, (k,exp) in zip(all_metric_dict, element_exp_names):

    openmc_result_dict[k][exp] = entry

Next we process the Fispact simulations results from the IAEA Conderc benchmarks so that they are ready to plot next to the OpenMC simulation results and the experimental benchmark results.

In [ ]:
def read_fispact_output(filepath):
    lines = open(filepath).readlines()
    # don't read empty lines
    read = False

    lines = [q for q in lines if q.strip()]
    step = 0
    # get header
    # usually the last # line
    for indx, line in enumerate(lines):
        if read:
            spl = line.strip().split()
            now_step = int(spl[0])
            assert(step +1 == now_step)
            step = now_step
            assert(len(spl) == len(col_names)), print(len(spl), len(col_names), '\n', col_names)
            for indx, val in enumerate(spl):
                key = col_names[indx]
                if key == 'step':
                    d[key].append(int(val))
                else:
                    d[key].append(float(val))
            continue
        if line[0] == '#' and lines[indx+1][0] != '#':
            # this line with the column names
            # terrible
            l = line.strip().split()[1:]
            indx = 0
            new_l = []
            while True:
                if indx >= len(l):
                    break
                if l[indx].isalpha():
                    if l[indx+1][-1] == 'm': # metastable
                        if l[indx+1][:-1].isnumeric():
                            new_l.append(l[indx]+l[indx+1])
                            indx += 2
                        else:
                            new_l.append(l[indx])
                            indx += 1
                    else:
                        if l[indx+1].isnumeric(): # metastable
                            new_l.append(l[indx]+l[indx+1])
                            indx += 2
                        else:
                            new_l.append(l[indx])
                            indx += 1
                else:
                    new_l.append(l[indx])
                    indx += 1
            d = {k:[] for k in new_l}
            read = True
            col_names = copy.deepcopy(new_l)
            continue
    return d


fispact_result_dict = {}
for k,l in experiments.items():
    fispact_result_dict[k] = {}
    for exp in l:
        output_path = here / k / f'TENDL-2017_{exp}.nuclides'
        fispact_result_dict[k][exp] = read_fispact_output(output_path.resolve())

We combine the Fispact results (which are per nuclide) so that we have the total values for decay heat.

In [ ]:
fispact_imp_nuclides = {}
for k,l in experiments.items():
    fispact_imp_nuclides[k] = {}
    for exp in l:
        tot = fispact_result_dict[k][exp]['Total']
        indices = [1, len(tot)//2, -1]
        fispact_imp_nuclides[k][exp] = {}
        for i in indices:
            td = {k:v[i] for k,v in fispact_result_dict[k][exp].items() if k not in ['step', 'time', 'uncert', 'Total']}
            td = {k:v for k,v in sorted(td.items(), key=lambda item:item[1], reverse=True)}
            fispact_imp_nuclides[k][exp][i] = td

We now have the OpenMC outputs, Fispact and experimental results in a convenient form  ready for plotting.

The next code block plots the results so that they can be compared.

In [ ]:
for k,l in openmc_result_dict.items():
    for exp in l:
        fispact = fispact_result_dict[k][exp] # results in watts per gram
        t_fispact = np.array(fispact['time']) * 365.25 * 60 * 24 # minutes
        fispact_microwatt_gram = np.array(fispact['Total']) * 1e6
        fispact_uncert = np.array(fispact['uncert'])  * 1e6

        # openmc 
        t_openmc = openmc_result_dict[k][exp]['mass']['meta_time_d']
        # t_openmc = np.diff(t_openmc)
        decay_indx = 1
        t0 = t_openmc[decay_indx]
        t_openmc = t_openmc[decay_indx:] - t0
        openmc = openmc_result_dict[k][exp]['decay_heat']['meta_total']
        mass = openmc_result_dict[k][exp]['mass']['meta_total']
        openmc = np.array(openmc) / np.array(mass)
        openmc = openmc[decay_indx:] * 1e6 # watts to microwatts
        # days to minutes
        t_openmc = t_openmc * (60*24)
        # add on 0
        plt.plot(t_openmc, openmc, label='OPENMC', marker='x', alpha=0.5)

        measured = exp_data_dict['data'][k][exp]
        t_measured = exp_data_dict['minutes'][k][exp]
        unc_measured = exp_data_dict['unc'][k][exp]
        # add irradiation time to t_measured
        t_measured = np.array(t_measured)

        for index,sorted_dict in fispact_imp_nuclides[k][exp].items():
            print(index, sorted_dict)

        if 'hour' not in exp:
            t_measured = t_measured / (60*24)
        # plt.errorbar(t_measured, measured, unc_measured, label='Measured', linestyle='--', marker='x')
        plt.fill_between(t_measured, measured-(3*unc_measured), measured+(3*unc_measured),
                         alpha=0.4, label='Measured')
        plt.errorbar(t_fispact, fispact_microwatt_gram, fispact_uncert, label='FISPACT', marker='o', alpha=0.5)
        # plt.plot(t_origen, origen, label='ORIGEN', marker='1', alpha=0.5)
        plt.yscale('log')
        plt.xlabel('Minutes')
        plt.ylabel(r'Specific heat [$\frac{\mu W}{g}$]')
        plt.legend()
        plt.grid()
        plt.title(k + ' (%s)' %(exp))
        plt.savefig(Path('docs') / f'{k}_{exp}.png')
        plt.close()

1 {'Re190': 2.65578e-09, 'Os192m': 6.99874e-10, 'Os191m': 5.97163e-10, 'Os189m': 4.29433e-10, 'W189': 7.5656e-11, 'Ir191m': 6.12702e-11, 'W185m': 3.74919e-11, 'Re188m': 2.69445e-11, 'Re191': 2.3879e-11, 'Os193': 1.54939e-11, 'Re190m': 1.48351e-11, 'Os191': 1.34387e-11, 'Re189': 7.4046e-12, 'Re188': 7.15575e-12, 'Os183': 4.38552e-12, 'W187': 3.32054e-12, 'Os185': 3.21157e-12, 'Os183m': 2.71664e-12, 'Re186': 1.22243e-13}
11 {'Re190': 7.47928e-10, 'Os191m': 5.94161e-10, 'Os189m': 4.2458e-10, 'Ir191m': 6.15163e-11, 'W189': 5.36045e-11, 'Re188m': 2.17745e-11, 'Re191': 1.5871e-11, 'Os193': 1.546e-11, 'Re190m': 1.45315e-11, 'Os191': 1.34905e-11, 'Re188': 7.63402e-12, 'Re189': 7.41704e-12, 'Os183': 4.3647e-12, 'W185m': 3.48857e-12, 'W187': 3.31135e-12, 'Os185': 3.21147e-12, 'Os183m': 2.69855e-12, 'Re186': 1.22153e-13, 'Os192m': 0.0}
-1 {'Os191m': 5.69284e-10, 'Os189m': 3.85553e-10, 'Ir191m': 6.34643e-11, 'Os193': 1.51749e-11, 'Os191': 1.39176e-11, 'Re190m': 1.21937e-11, 'Re190': 9.48276e-12, '